# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-2281ce61c4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Nobel laureates explorative search") 


Consider the following exploratory scenario:

>  Investigate the authors who won the Nobel award for literature and get information about the nationality of the winners and analyse their literary production



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q37922`   | Nobel Prize in Literature     | node |
| `wd:Q36180`   | writer        | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q3624078` | Sovereign state  | node |
| `wd:Q213678`  | Vatican Library  | node |


Also consider

```
?p wdt:P27 wd:Q38  . 
?p wdt:P106/wdt:P279* wd:Q36180  . 
```

is the BGP to retrieve all **italian authors**

The workload should


1. Identify the BGP for obtaining the nobel laureates and the nominees with their relevant attributes

2. Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.

3. Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, Iran, and China

4. How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy) (you may check if they have a doctoral advisor)?

5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT( ?p)
WHERE { 
?p wdt:P27 wd:Q38  . 
?p wdt:P106/wdt:P279* wd:Q36180  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '46967')]


1

**1 - Identify the BGP for obtaining the nobel laureates and the nominees with their relevant attributes**

I look for the entity "Nobel prize". I know that some italian authors won the nobel prize. I explore their properties

In [8]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 
?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

OPTIONAL{?p <http://schema.org/name> ?pName .}

} 
ORDER BY ASC (?pName)
LIMIT 1400
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8992'), ('pName', '1914-1918-Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3314'), ('pName', '365chess player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9046'), ('pName', 'A*dS Encyclopedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9634'), ('pName', 'AADFI member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P864'), ('pName', 'ACM Digital Library author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3332'), ('pName', 'ACM Digital Library citation ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3901'), ('pName', 'ADAGP artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4114'), ('pName', 'ADK member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5463'), ('pName', 'AE member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6038'), ('pName', 'AFAS author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2342'), ('pName', 'AGORHA person/institution ID')]
[('p', 'http://www.wikidata.org

1400

I've found two properties P1411 "nominated for" and "winner". They can be connected to nobel prize

In [10]:
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE { 
?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
FILTER (?p = wdt:P1411 || ?p = wdt:P1346)

} 
ORDER BY ASC (?pName)
LIMIT 1000
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q214665'), ('sName', 'Nino Rota'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q4376972'), ('oName', 'Grammy Award for Best Score Soundtrack for Visual Media')]
[('s', 'http://www.wikidata.org/entity/Q214665'), ('sName', 'Nino Rota'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q1027904'), ('oName', 'Grammy Award for Song of the Year')]
[('s', 'http://www.wikidata.org/entity/Q214665'), ('sName', 'Nino Rota'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q1422140'), ('oName', 'Golden Globe Award for Best Original Score')]
[('s', 'http://www.wikidata.org/entity/Q214665'), ('sName', 'Nino Rota'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q22235329'), 

435

In [ ]:
Change the previous query a bit.  

In [12]:
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE { 
?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
FILTER (?p = wdt:P1411 || ?p = wdt:P1346)

} 
ORDER BY DESC (?pName) ASC (?pName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q106032921'), ('sName', 'Massimo Cannoletta'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q16740537'), ('oName', "L'eredità")]
[('s', 'http://www.wikidata.org/entity/Q214665'), ('sName', 'Nino Rota'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q4376972'), ('oName', 'Grammy Award for Best Score Soundtrack for Visual Media')]
[('s', 'http://www.wikidata.org/entity/Q214665'), ('sName', 'Nino Rota'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q1027904'), ('oName', 'Grammy Award for Song of the Year')]
[('s', 'http://www.wikidata.org/entity/Q214665'), ('sName', 'Nino Rota'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q1422140'), ('oName', 'Golden Globe Award for B

435

"Winner" is not the correct property to use. I know that S. Quasimodo and some others persons won a nobel prize in literaure, but here I only see their nomination. I explore S. Quasimodo. 

In [14]:
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE { 
wd:Q83038 ?p ?o .

OPTIONAL{?p <http://schema.org/name> ?pName .}

OPTIONAL{?o <http://schema.org/name> ?oName .}

} 
ORDER BY DESC (?pName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('o', 'http://data.bibliotheken.nl/id/thes/p068659911')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('o', 'https://livedata.bibsys.no/authority/90079112')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1150'), ('o', 'http://rvk.uni-regensburg.de/nt/IV+41880')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1741'), ('o', 'http://data.beeldengeluid.nl/gtaa/144094')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('o', 'http://viaf.org/viaf/59097336')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163'), ('o', 'http://id.worldcat.org/fast/36582')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163'), ('o', 'http://id.worldcat.org/fast/478336')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('o', 'https://d-nb.info/gnd/118793802

186

P166 "award received" is the correct property. Modifying the previous query. 

In [16]:
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE { 
?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
FILTER (?p = wdt:P1411 || ?p = wdt:P166)

} 
ORDER BY DESC (?sName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q106057'), ('sName', 'Yves Montand'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q1967731'), ('oName', 'National Society of Film Critics Award for Best Actor')]
[('s', 'http://www.wikidata.org/entity/Q106057'), ('sName', 'Yves Montand'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q900494'), ('oName', 'César Award for Best Actor')]
[('s', 'http://www.wikidata.org/entity/Q106057'), ('sName', 'Yves Montand'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q3404523'), ('oName', 'David di Donatello for Best Foreign Actor')]
[('s', 'http://www.wikidata.org/entity/Q106057'), ('sName', 'Yves Montand'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q4377021'), 

1500

I try to restrict the results by filtering for "nobel" word with regex

In [8]:
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE { 
?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
FILTER (?p = wdt:P1411 || ?p = wdt:P166)
FILTER REGEX(?oName,".*[nN]obel.*")

} 
ORDER BY DESC (?sName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q558295'), ('sName', 'Vasco Pratolini'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q37922'), ('oName', 'Nobel Prize in Literature')]
[('s', 'http://www.wikidata.org/entity/Q2706353'), ('sName', 'Ugo Cerletti'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q80061'), ('oName', 'Nobel Prize in Physiology or Medicine')]
[('s', 'http://www.wikidata.org/entity/Q83038'), ('sName', 'Salvatore Quasimodo'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q37922'), ('oName', 'Nobel Prize in Literature')]
[('s', 'http://www.wikidata.org/entity/Q83038'), ('sName', 'Salvatore Quasimodo'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q37922'), ('oName', 'Nobel Prize i

55

Ok, the previous query works. Now I estract only the authors inolved in the Literature domanin

In [25]:
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE { 
?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
FILTER (?p = wdt:P1411 || ?p = wdt:P166)
FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
ORDER BY ASC (?sName) ASC (?pName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q161933'), ('sName', 'Alberto Moravia'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q37922'), ('oName', 'Nobel Prize in Literature')]
[('s', 'http://www.wikidata.org/entity/Q192348'), ('sName', 'Benedetto Croce'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q37922'), ('oName', 'Nobel Prize in Literature')]
[('s', 'http://www.wikidata.org/entity/Q312628'), ('sName', 'Carlo Levi'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q37922'), ('oName', 'Nobel Prize in Literature')]
[('s', 'http://www.wikidata.org/entity/Q765'), ('sName', 'Dario Fo'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q37922'), ('oName', 'Nobel Prize in Literature')]
[('s', 'http://

16

Not all authors that have won the Nobel have also the property "nominated for". 
The other informations that I can include are P569 date of birth; P570 date of death; P746 date of disappearence; P119 place of burial; P26 spouse. 


In [11]:
queryString = """
CONSTRUCT {
    ?s <http://schema.org/name> ?sName;
    wdt:P570 ?date2 ;
    wdt:P746 ?date3 ;
    wdt:P119 ?bur ;
    wdt:P26 ?wf .

}
WHERE { 
?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P569 ?date1 . #born
OPTIONAL {?s wdt:P570 ?date2 .} #death
OPTIONAL {?s wdt:P746 ?date3 .}
OPTIONAL {?s wdt:P119 ?bur .}
OPTIONAL {?s wdt:P26 ?wf .}

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
FILTER (?p = wdt:P1411 || ?p = wdt:P166)
FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
ORDER BY ASC (?sName)
"""

print("Results")
run_query(queryString)

Results


/home/ubuntu/miniconda3/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:979: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'CONSTRUCT' SPARQL query form
  warnings.warn("Sending Accept header '*/*' because unexpected returned format '%s' in a '%s' SPARQL query form" % (self.returnFormat, self.queryType), RuntimeWarning)


[('s', 'http://www.wikidata.org/entity/Q311802'), ('p', 'http://www.wikidata.org/prop/direct/P570'), ('o', '1970-06-02T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q192348'), ('p', 'http://schema.org/name'), ('o', 'Benedetto Croce')]
[('s', 'http://www.wikidata.org/entity/Q1751663'), ('p', 'http://www.wikidata.org/prop/direct/P570'), ('o', '1972-02-29T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q161933'), ('p', 'http://www.wikidata.org/prop/direct/P570'), ('o', '1990-09-26T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q161933'), ('p', 'http://schema.org/name'), ('o', 'Alberto Moravia')]
[('s', 'http://www.wikidata.org/entity/Q312628'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('o', 'http://www.wikidata.org/entity/Q16206')]
[('s', 'http://www.wikidata.org/entity/Q529858'), ('p', 'http://www.wikidata.org/prop/direct/P570'), ('o', '1956-07-08T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q161933'), ('p', 'http://www.wikidata.org/prop/direct/P26'), ('o'

42

I change the query to another easier form

In [53]:
queryString = """
SELECT DISTINCT ?sName AS ?Name ?date1 AS ?BornIn ?date2 AS ?DeadIn ?burName AS ?BuriedIn (GROUP_CONCAT(DISTINCT ?wfName ;separator=", ") AS ?Wifes)
WHERE { 

?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P569 ?date1 . 
OPTIONAL {?s wdt:P570 ?date2} 
OPTIONAL {?s wdt:P746 ?date3}
OPTIONAL {?s wdt:P119 ?bur .
?bur <http://schema.org/name> ?burName .}
OPTIONAL {?s wdt:P26 ?wf .
?wf <http://schema.org/name> ?wfName .}

OPTIONAL{?p <http://schema.org/name> ?pName}
OPTIONAL{?s <http://schema.org/name> ?sName}
OPTIONAL{?o <http://schema.org/name> ?oName}


FILTER (?p = wdt:P1411 || ?p = wdt:P166)
FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?sName ?burName ?date2 ?date1 ?s
ORDER BY ASC (?sName)
"""

print("Results")
run_query(queryString)

Results
[('Name', 'Alberto Moravia'), ('BornIn', '1907-11-28T00:00:00Z'), ('DeadIn', '1990-09-26T00:00:00Z'), ('BuriedIn', 'Campo Verano'), ('Wifes', 'Carmen Llera, Elsa Morante')]
[('Name', 'Benedetto Croce'), ('BornIn', '1866-02-25T00:00:00Z'), ('DeadIn', '1952-11-20T00:00:00Z'), ('Wifes', '')]
[('Name', 'Carlo Levi'), ('BornIn', '1902-11-29T00:00:00Z'), ('DeadIn', '1975-01-04T00:00:00Z'), ('BuriedIn', 'Province of Matera'), ('Wifes', '')]
[('Name', 'Dario Fo'), ('BornIn', '1926-03-24T00:00:00Z'), ('DeadIn', '2016-10-13T00:00:00Z'), ('BuriedIn', 'monumental cemetery of Milan'), ('Wifes', 'Franca Rame')]
[('Name', 'Elio Vittorini'), ('BornIn', '1908-07-23T00:00:00Z'), ('DeadIn', '1966-02-12T00:00:00Z'), ('Wifes', '')]
[('Name', 'Eugenio Montale'), ('BornIn', '1896-10-12T00:00:00Z'), ('DeadIn', '1981-09-12T00:00:00Z'), ('BuriedIn', 'Cemetery of San Felice a Ema'), ('Wifes', 'Drusilla Tanzi')]
[('Name', 'Giovanni Papini'), ('BornIn', '1881-01-09T00:00:00Z'), ('DeadIn', '1956-07-08T00:00

17

I'm not satisfied with the results. I would like to include info about the prize, Q37922. 

In [54]:
queryString = """
SELECT ?p ?pName
WHERE { 

wd:Q37922 ?p ?o .

OPTIONAL{?o <http://schema.org/name> ?oName}
OPTIONAL{?p <http://schema.org/name> ?pName}

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2581')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P8408')]
[('p', 'http://schema.org/name')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P3569'), ('pName', 'Cultureel Woordenboek ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3509'), ('pName', 'Dagens Nyheter topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9545'), ('pName', 'Encyclopedia of China Online Database ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID')]
[('p', 'http:

53

Explore the property "award received" P166

In [16]:
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE { 

wd:P166 ?p ?o .

OPTIONAL{?o <http://schema.org/name> ?oName}
OPTIONAL{?p <http://schema.org/name> ?pName}

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://wikiba.se/ontology#directClaim'), ('o', 'http://www.wikidata.org/prop/direct/P166'), ('oName', 'award received')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Property')]
[('p', 'http://schema.org/name'), ('o', 'award received')]
[('p', 'http://wikiba.se/ontology#claim'), ('o', 'http://www.wikidata.org/prop/P166')]
[('p', 'http://wikiba.se/ontology#novalue'), ('o', 'http://www.wikidata.org/prop/novalue/P166')]
[('p', 'http://wikiba.se/ontology#propertyType'), ('o', 'http://wikiba.se/ontology#WikibaseItem')]
[('p', 'http://wikiba.se/ontology#qualifier'), ('o', 'http://www.wikidata.org/prop/qualifier/P166')]
[('p', 'http://wikiba.se/ontology#qualifierValue'), ('o', 'http://www.wikidata.org/prop/qualifier/value/P166')]
[('p', 'http://wikiba.se/ontology#reference'), ('o', 'http://www.wikidata.org/prop/reference/P166')]
[('p', 'http://wikiba.se/ontology#referenceValue'), ('o', 'http://www.wikidata.org/prop/reference/value

45

In [57]:
queryString = """
SELECT ?p ?pName
WHERE { 

?s ?p wd:P166 .

OPTIONAL{?s <http://schema.org/name> ?sName}
OPTIONAL{?p <http://schema.org/name> ?pName}

} 
ORDER BY ASC (?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://schema.org/about')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this ty

29

Nothing useful from the last two queries.

**Answer 1** 

I decided to include the information regardindg the born date "P569", the death date "P570", the place where the person is buried in "P119". I also include the wifes.

```
SELECT DISTINCT ?sName AS ?Name ?date1 AS ?BornIn ?date2 AS ?DeadIn ?burName AS ?BuriedIn (GROUP_CONCAT(DISTINCT ?wfName ;separator=", ") AS ?Wifes)
WHERE { 

?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P569 ?date1 . 
OPTIONAL {?s wdt:P570 ?date2} 
OPTIONAL {?s wdt:P746 ?date3}
OPTIONAL {?s wdt:P119 ?bur .
?bur <http://schema.org/name> ?burName .}

OPTIONAL {?s wdt:P26 ?wf .
?wf <http://schema.org/name> ?wfName .}

OPTIONAL{?p <http://schema.org/name> ?pName}
OPTIONAL{?s <http://schema.org/name> ?sName}
OPTIONAL{?o <http://schema.org/name> ?oName}


FILTER (?p = wdt:P1411 || ?p = wdt:P166)
FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?sName ?burName ?date2 ?date1 ?s
ORDER BY ASC (?sName) 
```
-----------------------


**2 - Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.**

Using the previous query about the italian winner or nominee. Now I filter only for the winners

In [58]:
queryString = """
SELECT DISTINCT ?nation ?nName
WHERE { 
?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
FILTER (?p = wdt:P166)
FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
ORDER BY ASC (?nName) 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('nation', 'http://www.wikidata.org/entity/Q408'), ('nName', 'Australia')]
[('nation', 'http://www.wikidata.org/entity/Q40'), ('nName', 'Austria')]
[('nation', 'http://www.wikidata.org/entity/Q28513'), ('nName', 'Austria-Hungary')]
[('nation', 'http://www.wikidata.org/entity/Q184'), ('nName', 'Belarus')]
[('nation', 'http://www.wikidata.org/entity/Q31'), ('nName', 'Belgium')]
[('nation', 'http://www.wikidata.org/entity/Q129286'), ('nName', 'British India')]
[('nation', 'http://www.wikidata.org/entity/Q219'), ('nName', 'Bulgaria')]
[('nation', 'http://www.wikidata.org/entity/Q16'), ('nName', 'Canada')]
[('nation', 'http://www.wikidata.org/entity/Q298'), ('nName', 'Chile')]
[('nation', 'http://www.wikidata.org/entity/Q739'), ('nName', 'Colombia')]
[('nation', 'http://www.wikidata.org/entity/Q33946'), ('nName', 'Czechoslovakia')]
[('nation', 'http://www.wikidata.org/entity/Q35'), ('nName', 'Denmark')]
[('nation', 'http://www.wikidata.org/entity/Q79'), ('nName', 'Egypt')]
[('natio

66

First of all I can see that Iran doesn'thave any author who won the Nobel for the Literature. 
Now I count the winners 

In [62]:
queryString = """
SELECT ?nName (COUNT(DISTINCT ?s) AS ?num_winners) 
WHERE { 

?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}

FILTER (?p = wdt:P166)
FILTER (?nation = wd:Q142 || ?nation = wd:Q38)

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?nName
ORDER BY ASC (?nName) 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('nName', 'France'), ('num_winners', '17')]
[('nName', 'Italy'), ('num_winners', '3')]


2

I know that Grazia Deledda, won a nobel for literature. I try to find her identity. 

In [66]:
queryString = """
SELECT DISTINCT ?w ?wName
WHERE { 

?w wdt:P27 wd:Q38  . 
?w wdt:P106/wdt:P279* wd:Q36180  . 

?w ?p ?o .

?w <http://schema.org/name> ?wName .

FILTER REGEX(?wName,"Grazia ")

} 
"""

print("Results")
run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q15994759'), ('wName', 'Maria Grazia Spillantini')]
[('w', 'http://www.wikidata.org/entity/Q106540428'), ('wName', 'Maria Grazia Bonanno')]
[('w', 'http://www.wikidata.org/entity/Q108516060'), ('wName', 'Maria Grazia Fileni')]
[('w', 'http://www.wikidata.org/entity/Q50309018'), ('wName', 'Maria Grazia Ciani')]
[('w', 'http://www.wikidata.org/entity/Q108395013'), ('wName', 'Maria Grazia Albanesi')]
[('w', 'http://www.wikidata.org/entity/Q105478447'), ('wName', 'Maria Grazia Ceccarelli')]
[('w', 'http://www.wikidata.org/entity/Q105393510'), ('wName', 'Maria Grazia Lenisa')]
[('w', 'http://www.wikidata.org/entity/Q16561909'), ('wName', 'Grazia Livi')]
[('w', 'http://www.wikidata.org/entity/Q88446603'), ('wName', 'Grazia Papola')]
[('w', 'http://www.wikidata.org/entity/Q105711886'), ('wName', 'Maria Grazia Meriggi')]
[('w', 'http://www.wikidata.org/entity/Q106648231'), ('wName', 'Maria Grazia Folli')]
[('w', 'http://www.wikidata.org/entity/Q63

37

Maybe she is not considered a writer

In [31]:
queryString = """
SELECT DISTINCT ?w ?wName ?nat
WHERE { 

?w wdt:P27 ?nat.  #wd:Q38  . 

?w ?p ?o .

?w <http://schema.org/name> ?wName .

FILTER REGEX(?wName,"Grazia Deledda")

} 
ORDER BY ASC (?wName)
"""

print("Results")
run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q7728'), ('wName', 'Grazia Deledda'), ('nat', 'http://www.wikidata.org/entity/Q172579')]


1

She is present in the dataset but not listed in Italy

Try to find a date property. Explore the nobel prize

In [18]:
queryString = """
SELECT ?class ?cName
WHERE { 

wd:Q37922 ?p ?class  . 

?class <http://schema.org/name> ?cName .

?p <http://schema.org/name> ?pName .


} 

"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q6538'), ('cName', 'Günter Grass')]
[('class', 'http://www.wikidata.org/entity/Q392'), ('cName', 'Bob Dylan')]
[('class', 'http://www.wikidata.org/entity/Q272855'), ('cName', 'Kazuo Ishiguro')]
[('class', 'http://www.wikidata.org/entity/Q254032'), ('cName', 'Olga Tokarczuk')]
[('class', 'http://www.wikidata.org/entity/Q72334'), ('cName', 'Toni Morrison')]
[('class', 'http://www.wikidata.org/entity/Q131487'), ('cName', 'Karl Adolph Gjellerup')]
[('class', 'http://www.wikidata.org/entity/Q121180'), ('cName', 'Władysław Reymont')]
[('class', 'http://www.wikidata.org/entity/Q159552'), ('cName', 'Johannes V. Jensen')]
[('class', 'http://www.wikidata.org/entity/Q44107'), ('cName', 'Peter Handke')]
[('class', 'http://www.wikidata.org/entity/Q47755'), ('cName', 'Imre Kertész')]
[('class', 'http://www.wikidata.org/entity/Q44593'), ('cName', 'V. S. Naipaul')]
[('class', 'http://www.wikidata.org/entity/Q18143'), ('cName', 'Gao Xingjian')]
[('clas

25

Explore the property P166

In [23]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?person wdt:P166 ?award  . 

?award ?p ?o.

?p <http://schema.org/name> ?pName .


} 
ORDER BY ASC (?pName)

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4896'), ('pName', '3D model')]
[('p', 'http://www.wikidata.org/prop/direct/P8828'), ('pName', 'AAA attraction ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9084'), ('pName', 'ABC News topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2179'), ('pName', 'ACM Classification Code (2012)')]
[('p', 'http://www.wikidata.org/prop/direct/P7979'), ('pName', 'ACM conference ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7003'), ('pName', 'ACMI web ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6981'), ('pName', 'ACNP journal ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3901'), ('pName', 'ADAGP artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4024'), ('pName', 'ADW taxon ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3593'), ('pName', 'AFI Catalog of Feature Films ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2345'), ('pName', 'AGORHA event identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P234

1918

Maybe P585 "point in time" can help...

In [33]:
queryString = """
SELECT DISTINCT ?o 
WHERE { 

?person wdt:P166 ?award  . 

?award wdt:P585 ?o.

?person <http://schema.org/name> ?pName .

?award <http://schema.org/name> ?aName .

FILTER REGEX(?aName,".*[nN]obel.*")
FILTER REGEX(?aName,".*[Ll]iterature.*")
} 


"""

print("Results")
run_query(queryString)

Results
Empty


0

Last chance....

In [34]:
queryString = """
SELECT DISTINCT ?p ?prName ?o 
WHERE { 

?person wdt:P166 ?award  . 

?award ?p ?o.

?person  <http://schema.org/name> ?pName .
?p  <http://schema.org/name> ?prName .
?award <http://schema.org/name> ?aName .

FILTER REGEX(?aName,".*[nN]obel.*")
FILTER REGEX(?aName,".*[Ll]iterature.*")
} 


"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('prName', 'Gran Enciclopèdia Catalana ID'), ('o', '21812881')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q159552')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q254032')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q7241')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q392')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q44593')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q121180')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q47561')]
[('p', 'http:/

47

Hoping that "inception" 571 can help

In [44]:
queryString = """
SELECT DISTINCT ?person ?date
WHERE { 

?person wdt:P166 ?award  . 

?award wdt:P571 ?date.

?person  <http://schema.org/name> ?pName .

?award <http://schema.org/name> ?aName .

FILTER REGEX(?aName,".*[nN]obel.*")
FILTER REGEX(?aName,".*[Ll]iterature.*")

} 


"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q159552'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q254032'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q34743'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q5878'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q41042'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q57074'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q42122'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q80900'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q33760'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q7241'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.wikidata.org/entity/Q83184'), ('date', '1901-01-01T00:00:00Z')]
[('person', 'http://www.

118

In [ ]:
No results.

In [ ]:
Q7191

In [47]:
queryString = """
SELECT DISTINCT ?pr ?pName ?o ?oName
WHERE { 

?person ?p wd:Q7191  . 

wd:Q7191 ?pr ?o.

?pr  <http://schema.org/name> ?pName .

OPTIONAL{?o <http://schema.org/name> ?oName .}

} 


"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q44585'), ('oName', 'Nobel Prize in Chemistry')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19190511'), ('oName', 'New Encyclopedic Dictionary')]
[('pr', 'http://www.wikidata.org/prop/direct/P837'), ('pName', 'day in year for periodic occurrence'), ('o', 'http://www.wikidata.org/entity/Q2302'), ('oName', 'December 10')]
[('pr', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by'), ('o', 'http://www.wikidata.org/entity/Q23810'), ('oName', 'Alfred Nobel')]
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q80061'), ('oName', 'Nobel Prize in Physiology or Medicine')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q34'), ('oName', 'Sweden')]
[('

86

--------------

2 - Get the number of Italian, French and Iranian winners

France 17
Italy 3

```
SELECT ?nName (COUNT(DISTINCT ?s) AS ?num_winners) 
WHERE { 

?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}

FILTER (?p = wdt:P166)
FILTER (?nation = wd:Q142 || ?nation = wd:Q38)

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?nName
ORDER BY ASC (?nName) 
LIMIT 100

```
-----------------------

Now I try to information about the year of the prize. the "Nobel ID" has in the URL information about a year. I suspect that is the one in which the author won the prize

In [79]:
queryString = """
SELECT DISTINCT ?sName ?nName ?date
WHERE { 

?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .
?s wdt:P3188 ?year .

?nation <http://schema.org/name> ?nName .

BIND(REPLACE(STR(?year), "^(literature/laureates/)?(.*?)/.*", "$2") AS ?date)

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}

FILTER (?p = wdt:P166)

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

FILTER(str(?date) >= "2000" && ?date != "literature")

} 

ORDER BY ASC (?nName)
"""

print("Results")
run_query(queryString)

Results
[('sName', 'J. M. Coetzee'), ('nName', 'Australia'), ('date', '2003')]
[('sName', 'Elfriede Jelinek'), ('nName', 'Austria'), ('date', '2004')]
[('sName', 'Svetlana Aleksievich'), ('nName', 'Belarus'), ('date', '2015')]
[('sName', 'Alice Munro'), ('nName', 'Canada'), ('date', '2013')]
[('sName', 'Jean-Marie Gustave Le Clézio'), ('nName', 'France'), ('date', '2008')]
[('sName', 'Gao Xingjian'), ('nName', 'France'), ('date', '2000')]
[('sName', 'Patrick Modiano'), ('nName', 'France'), ('date', '2014')]
[('sName', 'Herta Müller'), ('nName', 'Germany'), ('date', '2009')]
[('sName', 'Imre Kertész'), ('nName', 'Hungary'), ('date', '2002')]
[('sName', 'Jean-Marie Gustave Le Clézio'), ('nName', 'Mauritius'), ('date', '2008')]
[('sName', 'Gao Xingjian'), ('nName', "People's Republic of China"), ('date', '2000')]
[('sName', 'Mo Yan'), ('nName', "People's Republic of China"), ('date', '2012')]
[('sName', 'Mario Vargas Llosa'), ('nName', 'Peru'), ('date', '2010')]
[('sName', 'Olga Tokarczuk

26

**Answer 2**

From this last complex but effective query I can say that UK won more awards in the Nobel Literature domanin

**3 - Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, Iran, and China**

I can re use the query for the previous point adding more countries

In [3]:
queryString = """
SELECT ?nName (COUNT(DISTINCT ?s) AS ?num_winners) 
WHERE { 

?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}

FILTER (?p = wdt:P166)
FILTER (?nation = wd:Q38 || ?nation = wd:Q183 || ?nation = wd:Q142 || ?nation = wd:Q218 || ?nation = wd:Q35 || ?nation = wd:Q148)

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?nName
ORDER BY DESC (?num_winners) 
"""

print("Results")
run_query(queryString)

Results
[('nName', 'France'), ('num_winners', '17')]
[('nName', 'Germany'), ('num_winners', '6')]
[('nName', 'Denmark'), ('num_winners', '4')]
[('nName', 'Italy'), ('num_winners', '3')]
[('nName', "People's Republic of China"), ('num_winners', '2')]
[('nName', 'Romania'), ('num_winners', '1')]


6

**Answer 3**

France is the learder of the list, with 17 awards! 

-------------------

**4 - How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy) (you may check if they have a doctoral advisor)?**

I use the property P184 "doctoral advisor"

In [11]:
queryString = """
SELECT DISTINCT (GROUP_CONCAT(DISTINCT ?nName ;separator="; ") AS ?Nation) ?sName ?phdadv ?phdadvName
WHERE { 

?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P184 ?phdadv .

?phdadv <http://schema.org/name> ?phdadvName .

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}

FILTER (?p = wdt:P166) 

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?sName ?phdadv ?phdadvName
"""

print("Results")
run_query(queryString)

Results
[('Nation', 'German Empire; Germany; Prussia; Weimar Republic'), ('sName', 'Rudolf Christoph Eucken'), ('phdadv', 'http://www.wikidata.org/entity/Q61280'), ('phdadvName', 'Friedrich Adolf Trendelenburg')]
[('Nation', 'United Kingdom; United Kingdom of Great Britain and Ireland'), ('sName', 'Bertrand Russell'), ('phdadv', 'http://www.wikidata.org/entity/Q183372'), ('phdadvName', 'Alfred North Whitehead')]


2

I try to use the property "academic degree" P512

In [12]:
queryString = """
SELECT DISTINCT (GROUP_CONCAT(DISTINCT ?nName ;separator="; ") AS ?Nation) ?sName ?adegree ?adName
WHERE { 

?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P512 ?adegree .

?adegree <http://schema.org/name> ?adName .

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}

FILTER (?p = wdt:P166) 

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?sName ?phdadv ?adegree ?adName
"""

print("Results")
run_query(queryString)

Results
[('Nation', 'Sweden'), ('sName', 'Erik Axel Karlfeldt'), ('adegree', 'http://www.wikidata.org/entity/Q2628227'), ('adName', 'Doctor of Sciences')]
[('Nation', 'Socialist Federal Republic of Yugoslavia'), ('sName', 'Ivo Andrić'), ('adegree', 'http://www.wikidata.org/entity/Q17281084'), ('adName', 'Doktor Nauk in Philosophy')]
[('Nation', 'Kingdom of Italy'), ('sName', 'Giosuè Carducci'), ('adegree', 'http://www.wikidata.org/entity/Q1054679'), ('adName', 'laurea')]
[('Nation', 'Denmark; German Empire; Prussia'), ('sName', 'Theodor Mommsen'), ('adegree', 'http://www.wikidata.org/entity/Q849697'), ('adName', 'doctorate')]
[('Nation', 'United Kingdom; United States of America'), ('sName', 'T.\xa0S. Eliot'), ('adegree', 'http://www.wikidata.org/entity/Q2628227'), ('adName', 'Doctor of Sciences')]
[('Nation', 'Poland'), ('sName', 'Olga Tokarczuk'), ('adegree', 'http://www.wikidata.org/entity/Q1589434'), ('adName', 'magister degree')]
[('Nation', 'France'), ('sName', 'Henri Bergson'), 

8

I have many variants of the specific degree that I'm trying to retrieve:

Doctor of Science-Doktor Nauk in Philosophy-doctorate

In [ ]:
Combining the two query

In [14]:
queryString = """
SELECT DISTINCT (GROUP_CONCAT(DISTINCT ?nName ;separator="; ") AS ?Nation) ?sName ?phd1 ?phd2
WHERE { 

?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

OPTIONAL{?s wdt:P184 ?phd1 .
    ?phd1 <http://schema.org/name> ?phd1Name .}

OPTIONAL{?s wdt:P512 ?phd2 .
    ?phd2 <http://schema.org/name> ?phd2Name .}

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}

?s <http://schema.org/name> ?sName .
?o <http://schema.org/name> ?oName .

FILTER (?p = wdt:P166) 

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")


FILTER (REGEX(?phd2Name,".*[Dd]octor.*"))

} 
GROUP BY ?sName ?phd1 ?phd2
"""

print("Results")
run_query(queryString)

Results
[('Nation', 'United Kingdom; United States of America'), ('sName', 'T.\xa0S. Eliot'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('Nation', 'Spain'), ('sName', 'Camilo José Cela'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('Nation', 'Denmark; German Empire; Prussia'), ('sName', 'Theodor Mommsen'), ('phd2', 'http://www.wikidata.org/entity/Q849697')]
[('Nation', 'France'), ('sName', 'Henri Bergson'), ('phd2', 'http://www.wikidata.org/entity/Q849697')]
[('Nation', 'Sweden'), ('sName', 'Erik Axel Karlfeldt'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]


5

It doesn't work properly

In [17]:
queryString = """
SELECT DISTINCT ?sName ?phd1 ?phd2
WHERE { 
{
?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P512 ?phd2 .
OPTIONAL{?phd2 <http://schema.org/name> ?phd2Name}

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}

?s <http://schema.org/name> ?sName .
?o <http://schema.org/name> ?oName .

FILTER (?p = wdt:P166) 

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")


FILTER (REGEX(?phd2Name,".*[Dd]octor.*"))
}
UNION
{
?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P184 ?phd1 .
OPTIONAL{?phd1 <http://schema.org/name> ?phd1Name .}

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}

?s <http://schema.org/name> ?sName .
?o <http://schema.org/name> ?oName .

FILTER (?p = wdt:P166) 

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

}

} 

"""

print("Results")
run_query(queryString)

Results
[('sName', 'T.\xa0S. Eliot'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('sName', 'Theodor Mommsen'), ('phd2', 'http://www.wikidata.org/entity/Q849697')]
[('sName', 'Camilo José Cela'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('sName', 'Henri Bergson'), ('phd2', 'http://www.wikidata.org/entity/Q849697')]
[('sName', 'Erik Axel Karlfeldt'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('sName', 'Bertrand Russell'), ('phd1', 'http://www.wikidata.org/entity/Q183372')]
[('sName', 'Rudolf Christoph Eucken'), ('phd1', 'http://www.wikidata.org/entity/Q61280')]


7

I left out one person. His degree is doKtor. I have to include that case

In [18]:
queryString = """
SELECT DISTINCT ?sName ?phd1 ?phd2
WHERE { 
{
?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P512 ?phd2 .
OPTIONAL{?phd2 <http://schema.org/name> ?phd2Name}

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}

?s <http://schema.org/name> ?sName .
?o <http://schema.org/name> ?oName .

FILTER (?p = wdt:P166) 

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")


FILTER (REGEX(?phd2Name,".*[Dd]o[ck]tor.*"))
}
UNION
{
?s wdt:P27 ?nation  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s wdt:P184 ?phd1 .
OPTIONAL{?phd1 <http://schema.org/name> ?phd1Name .}

?nation <http://schema.org/name> ?nName .

OPTIONAL{?p <http://schema.org/name> ?pName .}

?s <http://schema.org/name> ?sName .
?o <http://schema.org/name> ?oName .

FILTER (?p = wdt:P166) 

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

}

} 

"""

print("Results")
run_query(queryString)

Results
[('sName', 'T.\xa0S. Eliot'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('sName', 'Theodor Mommsen'), ('phd2', 'http://www.wikidata.org/entity/Q849697')]
[('sName', 'Ivo Andrić'), ('phd2', 'http://www.wikidata.org/entity/Q17281084')]
[('sName', 'Camilo José Cela'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('sName', 'Henri Bergson'), ('phd2', 'http://www.wikidata.org/entity/Q849697')]
[('sName', 'Erik Axel Karlfeldt'), ('phd2', 'http://www.wikidata.org/entity/Q2628227')]
[('sName', 'Bertrand Russell'), ('phd1', 'http://www.wikidata.org/entity/Q183372')]
[('sName', 'Rudolf Christoph Eucken'), ('phd1', 'http://www.wikidata.org/entity/Q61280')]


8

**Answer 4**

There are 8 Nobel winners in the literature field that have also a PhD. 

--------------------

**5 - Are there books from Literature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?**

Print all the properties of the objects connected to the writer

In [35]:
queryString = """
SELECT DISTINCT ?pr1 ?pr1Name
WHERE { 

?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

## property of the object connected to the writer

?book ?pr1 ?s . 

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
OPTIONAL{?pr1 <http://schema.org/name> ?pr1Name .}

FILTER (?p = wdt:P166)

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
ORDER BY ASC (?pr1Name)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr1', 'http://www.wikidata.org/prop/direct/P1877'), ('pr1Name', 'after a work by')]
[('pr1', 'http://www.wikidata.org/prop/direct/P50'), ('pr1Name', 'author')]
[('pr1', 'http://www.wikidata.org/prop/direct/P161'), ('pr1Name', 'cast member')]
[('pr1', 'http://www.wikidata.org/prop/direct/P971'), ('pr1Name', 'category combines topics')]
[('pr1', 'http://www.wikidata.org/prop/direct/P301'), ('pr1Name', "category's main topic")]
[('pr1', 'http://www.wikidata.org/prop/direct/P40'), ('pr1Name', 'child')]
[('pr1', 'http://www.wikidata.org/prop/direct/P825'), ('pr1Name', 'dedicated to')]
[('pr1', 'http://www.wikidata.org/prop/direct/P1889'), ('pr1Name', 'different from')]
[('pr1', 'http://www.wikidata.org/prop/direct/P108'), ('pr1Name', 'employer')]
[('pr1', 'http://www.wikidata.org/prop/direct/P22'), ('pr1Name', 'father')]
[('pr1', 'http://www.wikidata.org/prop/direct/P967'), ('pr1Name', 'guest of honor')]
[('pr1', 'http://www.wikidata.org/prop/direct/P2650'), ('pr1Name', 'interest

25

With propert P50 I can connect the book with the author. 

In [36]:
queryString = """
SELECT DISTINCT ?prb ?prbName
WHERE { 

?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

## property of the object connected to the writer

?book wdt:P50 ?s . 
?book ?prb ?obj . 

OPTIONAL{?p <http://schema.org/name> ?pName .}
OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}
OPTIONAL{?prb <http://schema.org/name> ?prbName .}

FILTER (?p = wdt:P166)

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
ORDER BY ASC (?prbName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('prb', 'http://www.wikidata.org/prop/direct/P8294'), ('prbName', 'AusStage work ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P6706'), ('prbName', 'De Agostini ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P1417'), ('prbName', 'Encyclopædia Britannica Online ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P3219'), ('prbName', 'Encyclopædia Universalis ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P5935'), ('prbName', 'Flanders Arts Institute production ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P646'), ('prbName', 'Freebase ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P2969'), ('prbName', 'Goodreads version/edition ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P8383'), ('prbName', 'Goodreads work ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P2671'), ('prbName', 'Google Knowledge Graph ID')]
[('prb', 'http://www.wikidata.org/prop/direct/P212'), ('prbName', 'ISBN-13')]
[('prb', 'http://www.wikidata.org/prop/direct/P269'), ('prb

63

These are the properties that connect an author to the Vatican library: 

P1017 Vatican Library ID (former scheme)
P8034 Vatican Library VcBA ID

but I want the connection of the book to the vatica library. I try to find it in the next queries

Here I consider the italian authors only

In [38]:
queryString = """
SELECT DISTINCT ?pr2 ?pr2Name
WHERE { 

?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 

?book wdt:P50 ?s .
?book ?pr2 ?obj .

OPTIONAL{?pr2 <http://schema.org/name> ?pr2Name .}

} 
ORDER BY ASC (?pr2Name)
LIMIT 500
"""

print("Results")
run_query(queryString)

Results
[('pr2', 'http://www.wikidata.org/prop/direct/P3332'), ('pr2Name', 'ACM Digital Library citation ID')]
[('pr2', 'http://www.wikidata.org/prop/direct/P6981'), ('pr2Name', 'ACNP journal ID')]
[('pr2', 'http://www.wikidata.org/prop/direct/P819'), ('pr2Name', 'ADS bibcode')]
[('pr2', 'http://www.wikidata.org/prop/direct/P6151'), ('pr2Name', 'ANICA ID')]
[('pr2', 'http://www.wikidata.org/prop/direct/P7896'), ('pr2Name', 'Academia.edu publication ID')]
[('pr2', 'http://www.wikidata.org/prop/direct/P1661'), ('pr2Name', 'Alexa rank')]
[('pr2', 'http://www.wikidata.org/prop/direct/P1562'), ('pr2Name', 'AllMovie title ID')]
[('pr2', 'http://www.wikidata.org/prop/direct/P1265'), ('pr2Name', 'AlloCiné film ID')]
[('pr2', 'http://www.wikidata.org/prop/direct/P5530'), ('pr2Name', 'Altmetric ID')]
[('pr2', 'http://www.wikidata.org/prop/direct/P5749'), ('pr2Name', 'Amazon Standard Identification Number')]
[('pr2', 'http://www.wikidata.org/prop/direct/P8586'), ('pr2Name', 'Anais do Museu Paulis

420

Using the Vatican city entity provided at the beginning of the notebook. "wd:Q213678". Here I show the properties that connect an object to the vatican library

In [39]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?s ?p wd:Q213678 .

OPTIONAL{?p <http://schema.org/name> ?pName .}

} 
ORDER BY ASC (?pName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('pName', 'affiliation')]
[('p', 'http://www.wikidata.org/prop/direct/P485'), ('pName', 'archives at')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P2378'), ('pName', 'issued by')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P189'), ('pName', 'location of discovery')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subje

20

Try to find book entity code from Savatore Quasimodo or Giuseppe Ungaretti Q311802 works

In [41]:
queryString = """
SELECT DISTINCT ?obj ?oName ?p ?pName 
WHERE { 

?obj ?p wd:Q83038 .


OPTIONAL{?p <http://schema.org/name> ?pName .}

OPTIONAL{?obj <http://schema.org/name> ?oName .}

} 
ORDER BY DESC (?pName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q16658518'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('obj', 'http://www.wikidata.org/entity/Q12825666'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('obj', 'https://www.wikidata.org/wiki/Special:EntityData/Q83038'), ('p', 'http://schema.org/about')]
[('obj', 'http://www.wikidata.org/entity/Q1248538'), ('oName', 'Barabbas'), ('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3981080'), ('oName', 'Tarantula'), ('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q56669227'), ('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3927780'), ('p', 'http://www.wikidata.org/prop/direct/P58'), ('pName', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q52923312'), ('oName', 'Salvatore Quasimodo School'), ('p', 'http://www.wikida

25

In [42]:
queryString = """
SELECT DISTINCT ?pr ?prName ?t ?tName 
WHERE { 

?obj ?p wd:Q83038 .
?obj ?pr ?t .

OPTIONAL{?pr <http://schema.org/name> ?prName .}

OPTIONAL{?t <http://schema.org/name> ?tName .}

} 
ORDER BY DESC (?pName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('prName', 'relative'), ('t', 'http://www.wikidata.org/entity/Q520812'), ('tName', 'Claude Frollo')]
[('pr', 'http://www.wikidata.org/prop/direct/P1040'), ('prName', 'film editor'), ('t', 'http://www.wikidata.org/entity/Q3608313'), ('tName', 'Alberto Gallitti')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('prName', 'medical condition'), ('t', 'http://www.wikidata.org/entity/Q737460'), ('tName', 'visual impairment')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('prName', 'medical condition'), ('t', 'http://www.wikidata.org/entity/Q12133'), ('tName', 'deafness')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('prName', 'medical condition'), ('t', 'http://www.wikidata.org/entity/Q1179623'), ('tName', 'physical disability')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('prName', 'medical condition'), ('t', 'http://www.wikidata.org/entity/Q478389'), ('tName', 'kyphosis')]
[('pr', 'http://www

344

In [48]:
queryString = """
SELECT DISTINCT ?pr ?prName ?t ?tName
WHERE { 

?obj ?p wd:Q311802 .
?obj ?pr ?t .

OPTIONAL{?pr <http://schema.org/name> ?prName .}
OPTIONAL{?t <http://schema.org/name> ?tName .}

} 
ORDER BY DESC (?prName)
LIMIT 1500
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.w3.org/2002/07/owl#sameAs'), ('t', 'http://www.wikidata.org/entity/Q311802'), ('tName', 'Giuseppe Ungaretti')]
[('pr', 'http://schema.org/about'), ('t', 'http://www.wikidata.org/entity/Q311802'), ('tName', 'Giuseppe Ungaretti')]
[('pr', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('t', 'http://wikiba.se/ontology#Item')]
[('pr', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('t', 'http://schema.org/Dataset')]
[('pr', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('t', 'http://data.bibliotheken.nl/id/thes/p068604831')]
[('pr', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('t', 'http://data.bibliotheken.nl/id/thes/p069814120')]
[('pr', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('t', 'http://data.bibliotheken.nl/id/thes/p070347433')]
[('pr', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('t', 'http://data.bibliotheken.nl/id/thes/p070256217')]
[('pr', 'http://www.wikidata.org/prop/direct-normalized/P1

417

The previous query did not produce any useful information. 

I use the entity "literary work" Q7725634 to search for a connection to Vatican Library

In [50]:
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q7725634 ?p wd:Q213678 .

OPTIONAL{?p <http://schema.org/name> ?pName .}

} 
ORDER BY ASC (?pName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Here I search for the class of the object connected to the Vat Library

In [51]:
queryString = """
SELECT DISTINCT ?type ?tName
WHERE { 

?obj ?p wd:Q213678 .

?obj wdt:P31 ?type .

OPTIONAL{?type <http://schema.org/name> ?tName .}

} 
ORDER BY ASC (?tName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q1754581'), ('tName', 'Evangeliary')]
[('type', 'http://www.wikidata.org/entity/Q690851'), ('tName', 'Gospel Book')]
[('type', 'http://www.wikidata.org/entity/Q1114838'), ('tName', 'Legendarium')]
[('type', 'http://www.wikidata.org/entity/Q46127'), ('tName', 'Menologium')]
[('type', 'http://www.wikidata.org/entity/Q60457486'), ('tName', 'Wikidata property for a discontinued website')]
[('type', 'http://www.wikidata.org/entity/Q96776953'), ('tName', 'Wikidata property for authority control (libraries)')]
[('type', 'http://www.wikidata.org/entity/Q19595382'), ('tName', 'Wikidata property for authority control for people')]
[('type', 'http://www.wikidata.org/entity/Q64263681'), ('tName', 'Wikidata property related to Catholicism and the Catholic Church')]
[('type', 'http://www.wikidata.org/entity/Q21745557'), ('tName', 'Wikidata property to identify organizations')]
[('type', 'http://www.wikidata.org/entity/Q4167836'), ('tName', 'Wikimedia

41

With the query about the authors and this previous one I look for the connection Book-Library printing the authors' works

In [56]:
queryString = """
SELECT DISTINCT ?obj1 ?obj1Name ?p2 ?p2Name
WHERE { 

?s wdt:P27 wd:Q38  . 
?s wdt:P106/wdt:P279* wd:Q36180  . 
?s ?p ?o .

?s ?p1 ?obj1 .

## property of the object connected to the writer

?obj1 ?p2 wd:Q213678 . 

OPTIONAL{?obj1 <http://schema.org/name> ?obj1Name .}

OPTIONAL{?p1 <http://schema.org/name> ?p1Name .}
OPTIONAL{?p2 <http://schema.org/name> ?p2Name .}

OPTIONAL{?s <http://schema.org/name> ?sName .}
OPTIONAL{?o <http://schema.org/name> ?oName .}


FILTER (?p = wdt:P166)

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 

LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Unfortuatel I did not retrive anything. Starting from the book and going back?

Literary work Q7725634

In [59]:
queryString = """
SELECT DISTINCT ?w ?wName ?oName ?obj
WHERE { 

?obj ?p wd:Q213678 .

?obj wdt:P31 wd:Q7725634 . 
#?w wdt:P50 ?obj .

#OPTIONAL{?w <http://schema.org/name> ?wName .}
OPTIONAL{?obj <http://schema.org/name> ?oName .}

} 
ORDER BY ASC (?wName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('oName', 'Indiculus superstitionum et paganiarum'), ('obj', 'http://www.wikidata.org/entity/Q916549')]
[('oName', 'De arte venandi cum avibus'), ('obj', 'http://www.wikidata.org/entity/Q1180580')]
[('oName', 'Lorsch Bee Blessing'), ('obj', 'http://www.wikidata.org/entity/Q956156')]


3

Trying to get the general class of the object inside vatican library. The previous query was too strict

In [11]:
queryString = """
SELECT DISTINCT ?type ?tName
WHERE { 

?obj ?p wd:Q213678 .

?obj wdt:P31/wdt:P279* ?type .

OPTIONAL{?type <http://schema.org/name> ?tName .}

} 
ORDER BY ASC (?tName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q1754581'), ('tName', 'Evangeliary')]
[('type', 'http://www.wikidata.org/entity/Q690851'), ('tName', 'Gospel Book')]
[('type', 'http://www.wikidata.org/entity/Q19899872'), ('tName', 'Greek vases')]
[('type', 'http://www.wikidata.org/entity/Q1114838'), ('tName', 'Legendarium')]
[('type', 'http://www.wikidata.org/entity/Q15474042'), ('tName', 'MediaWiki page')]
[('type', 'http://www.wikidata.org/entity/Q46127'), ('tName', 'Menologium')]
[('type', 'http://www.wikidata.org/entity/Q18616576'), ('tName', 'Wikidata property')]
[('type', 'http://www.wikidata.org/entity/Q64830420'), ('tName', 'Wikidata property associated with people')]
[('type', 'http://www.wikidata.org/entity/Q84764641'), ('tName', 'Wikidata property associated with websites')]
[('type', 'http://www.wikidata.org/entity/Q60457486'), ('tName', 'Wikidata property for a discontinued website')]
[('type', 'http://www.wikidata.org/entity/Q19847637'), ('tName', 'Wikidata property for 

100

Finally I've found the BOOK entity Q571. I search for the property that connect those obect to the Vatican Library

In [7]:
queryString = """
SELECT DISTINCT ?p ?pName ?tName
WHERE { 

?obj ?p wd:Q213678 .

?obj wdt:P31/wdt:P279* ?type .

OPTIONAL {?p <http://schema.org/name> ?pName .}
OPTIONAL{?type <http://schema.org/name> ?tName .}



} 
ORDER BY ASC (?tName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'Evangeliary')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('tName', 'Evangeliary')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'Gospel Book')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'Greek vases')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'Legendarium')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('tName', 'Legendarium')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic"), ('tName', 'MediaWiki page')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics'), ('tName', 'MediaWiki page')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'Menologium')]
[('p', 'http://www.wikidata.org/prop/di

100

Filtering for BOOK

In [9]:
queryString = """
SELECT DISTINCT ?p ?pName ?tName
WHERE { 

?obj ?p wd:Q213678 .

?obj wdt:P31/wdt:P279* ?type .

OPTIONAL {?p <http://schema.org/name> ?pName .}
OPTIONAL{?type <http://schema.org/name> ?tName .}

FILTER REGEX(?tName, "book")

} 
ORDER BY ASC (?tName)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'book')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('tName', 'book')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'handbook')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('tName', 'handbook')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'liturgical book')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('tName', 'liturgical book')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('tName', 'prayer book')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('tName', 'prayer book')]


8

I've discovered that I can use the properties "location" P276 or "collection" P195 . 

In [22]:
queryString = """
SELECT DISTINCT ?objName ?wName
WHERE { 

?writer wdt:P106/wdt:P279* wd:Q36180  . 
?writer wdt:P166 ?o .

?writer <http://schema.org/name> ?wName .

?obj wdt:P50 ?writer .

?obj wdt:P31/wdt:P279* wd:Q571 .
?obj <http://schema.org/name> ?objName .

FILTER NOT EXISTS {?obj wdt:P276 wd:Q213678 .}

?o <http://schema.org/name> ?oName .

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
ORDER BY ASC (?wName) 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('objName', 'One Day in the Life of Ivan Denisovich'), ('wName', 'Aleksandr Solzhenitsyn')]
[('objName', 'In the First Circle'), ('wName', 'Aleksandr Solzhenitsyn')]
[('objName', 'Monsieur Bergeret a Paris'), ('wName', 'Anatole France')]
[('objName', 'Les dieux ont soif'), ('wName', 'Anatole France')]
[('objName', "L'île des pingouins"), ('wName', 'Anatole France')]
[('objName', "L'anneau d'améthyste"), ('wName', 'Anatole France')]
[('objName', 'La rôtisserie de la reine Pédauque'), ('wName', 'Anatole France')]
[('objName', "L'Étui de nacre"), ('wName', 'Anatole France')]
[('objName', "L' orme du mail : Histoire contemporaine / Anatole France"), ('wName', 'Anatole France')]
[('objName', 'Les opinions de M. Jérôme Coignard / recueillies par Jacques Tournebroche ; et publiées par Anatole France'), ('wName', 'Anatole France')]
[('objName', "Le mannequin d'osier"), ('wName', 'Anatole France')]
[('objName', 'Si le grain ne meurt'), ('wName', 'André Gide')]
[('objName', 'Les caves d

100

Yes! There are many book not present in the Vatican Library

In [33]:
queryString = """
SELECT DISTINCT ?wName (COUNT(DISTINCT ?objName) AS ?num_booksNotInVL)
WHERE { 

?writer wdt:P106/wdt:P279* wd:Q36180  . 
?writer wdt:P166 ?o .

?writer <http://schema.org/name> ?wName .

?obj wdt:P50 ?writer .

?obj wdt:P31/wdt:P279* wd:Q571 .
?obj <http://schema.org/name> ?objName .

FILTER NOT EXISTS {?obj wdt:P276 wd:Q213678 .}

?o <http://schema.org/name> ?oName .

FILTER REGEX(?oName,".*[nN]obel.*")
FILTER REGEX(?oName,".*[Ll]iterature.*")

} 
GROUP BY ?wName
ORDER BY DESC (?wName) 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('wName', 'Yasunari Kawabata'), ('num_booksNotInVL', '11')]
[('wName', 'Wole Soyinka'), ('num_booksNotInVL', '14')]
[('wName', 'William Faulkner'), ('num_booksNotInVL', '1')]
[('wName', 'William Butler Yeats'), ('num_booksNotInVL', '1')]
[('wName', 'Toni Morrison'), ('num_booksNotInVL', '5')]


5

**Answer 5** 

Yasunari Kawabata is the author with the highest number of book non in VL